In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_4006/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/13 09:37:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/13 09:37:24 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for the Users table
users_data = {
    'user_id': [1, 2, 3, 4],
    'join_date': ['2019-01-01', '2019-02-09', '2019-01-19', '2019-05-21'],
    'favorite_brand': ['Lenovo', 'Samsung', 'LG', 'HP']
}

# Sample data for the Orders table
orders_data = {
    'order_id': [1, 2, 3, 4, 5, 6],
    'order_date': ['2019-08-01', '2019-08-02', '2019-08-03', '2019-08-04', '2019-08-04', '2019-08-05'],
    'item_id': [4, 2, 3, 1, 1, 2],
    'buyer_id': [1, 1, 2, 4, 3, 2],
    'seller_id': [2, 3, 3, 2, 4, 4]
}

# Sample data for the Items table
items_data = {
    'item_id': [1, 2, 3, 4],
    'item_brand': ['Samsung', 'Lenovo', 'LG', 'HP']
}

# Creating pandas DataFrames for Users, Orders, and Items tables
users_df = pd.DataFrame(users_data)
orders_df = pd.DataFrame(orders_data)
items_df = pd.DataFrame(items_data)

# Displaying the Users, Orders, and Items DataFrames
print("Users table:")
print(users_df)
print("\nOrders table:")
print(orders_df)
print("\nItems table:")
print(items_df)



df_person = spark.createDataFrame(users_df)
df_person.createOrReplaceTempView("Users")

df_person = spark.createDataFrame(orders_df)
df_person.createOrReplaceTempView("Orders")

df_person = spark.createDataFrame(items_df)
df_person.createOrReplaceTempView("Items")



Users table:
   user_id   join_date favorite_brand
0        1  2019-01-01         Lenovo
1        2  2019-02-09        Samsung
2        3  2019-01-19             LG
3        4  2019-05-21             HP

Orders table:
   order_id  order_date  item_id  buyer_id  seller_id
0         1  2019-08-01        4         1          2
1         2  2019-08-02        2         1          3
2         3  2019-08-03        3         2          3
3         4  2019-08-04        1         4          2
4         5  2019-08-04        1         3          4
5         6  2019-08-05        2         2          4

Items table:
   item_id item_brand
0        1    Samsung
1        2     Lenovo
2        3         LG
3        4         HP


In [32]:
query = """
    with t as (
        select
            *,
            case
                when rank_sell = 2 and favorite_brand = item_brand then 'yes'
                else 'no'
            end 2nd_item_fav_brand 
        from (
            select
                o.seller_id,
                o.order_date,
                u.favorite_brand,
                i.item_brand,
                row_number() over (partition by o.seller_id order by order_date asc) as rank_sell
            from
                Orders o
                left join Users u on o.seller_id = u.user_id
                left join Items i on o.item_id = i.item_id
            order by o.seller_id asc, o.order_date asc
        )
    ),
    
    k as (select distinct seller_id, 2nd_item_fav_brand from t where 2nd_item_fav_brand = 'yes')

    select distinct seller_id, 2nd_item_fav_brand from t where 2nd_item_fav_brand != 'yes' and seller_id not in (select seller_id from k) union all select * from k

"""

In [46]:
query_1 = """
    with o as (select
        o.seller_id,
        o.order_date,
        o.item_id,
        row_number() over (partition by o.seller_id order by o.order_date asc) as rk
    from Orders o)
    
    select
        u.user_id,
        case
            when u.favorite_brand = i.item_brand then 'yes'
            else 'no'
        end as 2nd_item_fav_brand   
    from Users u
    left join o on u.user_id = o.seller_id and o.rk = 2
    left join Items i on o.item_id = i.item_id

"""

In [47]:
spark.sql(query_1).show()

+-------+------------------+
|user_id|2nd_item_fav_brand|
+-------+------------------+
|      1|                no|
|      2|               yes|
|      3|               yes|
|      4|                no|
+-------+------------------+

